# PCA on embeddings


In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
#pgadmin
import os

import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import datetime

## Get id, which is used to match with the embedding array's index

In [4]:
repo_data_key = pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/clean_github_repos_157k.csv")
repo_data_key = repo_data_key[repo_data_key.description.notnull()]

In [5]:
repo_data_key["id"] = range(len(repo_data_key))

repo_data_key = repo_data_key[["slug", "id"]]
print(repo_data_key)

print(repo_data_key[repo_data_key.slug == "tensorflow/tensorflow"])

                                slug      id
0                          vuejs/vue       0
1                     facebook/react       1
2              tensorflow/tensorflow       2
3                     twbs/bootstrap       3
4                    ohmyzsh/ohmyzsh       4
...                              ...     ...
157533    VeryLittleGravitas/CDTADPQ  155806
157534  dajinchu/kde-connect-android  155807
157535   LibrinnoTeam/LibraryHelpBot  155808
157536               Twissi/Animator  155809
157537    WisapeAgency/WisapeAndroid  155810

[155811 rows x 2 columns]
                    slug  id
2  tensorflow/tensorflow   2


## Get Embeddings as array on 155,811 english repo description

In [6]:
embeddings = np.load('/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_description_embeddings.npy')

In [7]:
embeddings.shape

(155811, 768)

# PCA on bert classified repo description embeddings

We just want to get a list of repo (slugs) that are classified into a software type. We don't need to know which software type they are classified into. 

In [2]:
repo_label= pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/github_repos_157k_label.csv")
repo_label = repo_label[repo_label.bert == True]
repo_label = repo_label[["slug", "bert"]]

In [3]:
repo_label = repo_label.drop_duplicates(subset="slug") #de-duplicate by name—> unique names in the data


In [4]:
repo_label[repo_label.slug == "tensorflow/tensorflow"]

,slug,bert
2,tensorflow/tensorflow,True


In [5]:
repo_label

,slug,bert
2,tensorflow/tensorflow,True
25,tensorflow/models,True
68,huggingface/transformers,True
134,tiangolo/fastapi,True
159,pandas-dev/pandas,True
...,...,...
1410026,eric-erki/python-openid,True
1410548,cerha/pytis,True
1411254,maximilianduell/poppler,True
1411720,xieta/dev,True


In [6]:
#repo_label.to_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_classified_slugs.csv", index = False)



In [12]:
# get the array id
repo_label_id = pd.merge(repo_label, repo_data_key, on = "slug", how = "left")

print(len(repo_label_id))
repo_label_id[repo_label_id.slug == "tensorflow/tensorflow"]


22466


,slug,bert,id
0,tensorflow/tensorflow,True,2


In [13]:
repo_ids = repo_label_id["id"].tolist()
len(repo_ids)

22466

In [14]:
repo_label_embeddings = []
for repo_id in repo_ids:
    embedding = embeddings[repo_id]
    repo_label_embeddings.append(embedding)
len(repo_label_embeddings)   

22466

### check variance explained for top 20 pc's

In [16]:
pca = PCA(n_components=10)
pca.fit(repo_label_embeddings)
#percent of variance explained in each pc
pca.explained_variance_

array([0.59182804, 0.44056011, 0.35276217, 0.25366339, 0.24145401,
       0.20515416, 0.18419258, 0.16567654, 0.14366009, 0.12460496])

In [17]:
pca = PCA(n_components=3)
pca_embedding = pca.fit_transform(repo_label_embeddings)

In [18]:
pca_embedding.shape

(22466, 3)

In [19]:
embedding_df = pd.DataFrame({'x': pca_embedding[:, 0], 'y': pca_embedding[:, 1], 'z' : pca_embedding[:,2]})

In [20]:
embedding_df

,x,y,z
0,-0.302585,0.542825,0.254508
1,-0.067079,1.128374,0.166578
2,-0.180961,1.030554,0.416845
3,-0.577798,0.474864,-0.327835
4,-0.732955,1.453041,0.172779
...,...,...,...
22461,-0.349324,0.720493,0.400368
22462,0.045109,1.148794,-0.086775
22463,-0.492608,0.567995,0.216712
22464,0.368163,0.493885,0.198109


In [21]:
repo_label_id["x"] = embedding_df.x
repo_label_id["y"] = embedding_df.y
repo_label_id["z"] = embedding_df.z

In [22]:
repo_label_id

,slug,bert,id,x,y,z
0,tensorflow/tensorflow,True,2,-0.302585,0.542825,0.254508
1,tensorflow/models,True,25,-0.067079,1.128374,0.166578
2,huggingface/transformers,True,68,-0.180961,1.030554,0.416845
3,tiangolo/fastapi,True,134,-0.577798,0.474864,-0.327835
4,pandas-dev/pandas,True,159,-0.732955,1.453041,0.172779
...,...,...,...,...,...,...
22461,eric-erki/python-openid,True,153090,-0.349324,0.720493,0.400368
22462,cerha/pytis,True,153596,0.045109,1.148794,-0.086775
22463,maximilianduell/poppler,True,154290,-0.492608,0.567995,0.216712
22464,xieta/dev,True,154744,0.368163,0.493885,0.198109


In [27]:
repo_data_out = repo_label_id[repo_label_id.x.notnull()]
repo_data_out = repo_data_out[["slug", "x", "y", "z"]]
repo_data_out

,slug,x,y,z
0,tensorflow/tensorflow,-0.302585,0.542825,0.254508
1,tensorflow/models,-0.067079,1.128374,0.166578
2,huggingface/transformers,-0.180961,1.030554,0.416845
3,tiangolo/fastapi,-0.577798,0.474864,-0.327835
4,pandas-dev/pandas,-0.732955,1.453041,0.172779
...,...,...,...,...
22461,eric-erki/python-openid,-0.349324,0.720493,0.400368
22462,cerha/pytis,0.045109,1.148794,-0.086775
22463,maximilianduell/poppler,-0.492608,0.567995,0.216712
22464,xieta/dev,0.368163,0.493885,0.198109


In [28]:
#repo_data_out.to_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_classified_embedding_3d_pca.csv", index = False)

# PCA on blockchain/dataviz repo description embeddings

In [6]:
repo_blockchain_dataviz = pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_blockchain_dataviz.csv")
repo_blockchain_dataviz

,slug,software_type
0,ethereum/go-ethereum,is_blockchain
1,freqtrade/freqtrade,is_blockchain
2,btcsuite/btcd,is_blockchain
3,bitpay/bitcore,is_blockchain
4,bitcoinj/bitcoinj,is_blockchain
...,...,...
246,theJensenLab/phylogician,is_dataviz
247,davidmtech/old-browser-backup,is_dataviz
248,ajoposor/aoPlotlyAddOn,is_dataviz
249,saramic/data-fe,is_dataviz


In [17]:
repo_blockchain_dataviz_id = pd.merge(repo_blockchain_dataviz, repo_data, on = "slug")
repo_ids = repo_blockchain_dataviz_id["id"]
repo_ids

,slug,software_type,description,id
0,ethereum/go-ethereum,is_blockchain,Official Go implementation of the Ethereum pro...,151
1,freqtrade/freqtrade,is_blockchain,Free open source crypto trading bot,1080
2,btcsuite/btcd,is_blockchain,An alternative full node bitcoin implementatio...,2664
3,bitpay/bitcore,is_blockchain,A full stack for bitcoin and blockchain based ...,2854
4,bitcoinj/bitcoinj,is_blockchain,A library for working with Bitcoin,2876
...,...,...,...,...
246,theJensenLab/phylogician,is_dataviz,Our approach to visualization,123665
247,davidmtech/old-browser-backup,is_dataviz,JavaScript WebGL 3D maps rendering engine,125701
248,ajoposor/aoPlotlyAddOn,is_dataviz,a Library of functions to be used to add funct...,126660
249,saramic/data-fe,is_dataviz,a front end for data visualisation experiments,131080


In [20]:
print(repo_data.iloc[151])

slug                                        ethereum/go-ethereum
description    Official Go implementation of the Ethereum pro...
id                                                           151
Name: 151, dtype: object


In [15]:
repo_blockchain_dataviz_embeddings = []
for repo_id in repo_ids:
    embedding = embeddings[repo_id]
    repo_blockchain_dataviz_embeddings.append(embedding)
    

In [21]:

pca = PCA(n_components=3)
pca_embedding = pca.fit_transform(repo_blockchain_dataviz_embeddings)

In [22]:
embedding_df = pd.DataFrame({'x': pca_embedding[:, 0], 'y': pca_embedding[:, 1], 'z' : pca_embedding[:,2]})

In [23]:
repo_blockchain_dataviz_id["x"] = embedding_df.x
repo_blockchain_dataviz_id["y"] = embedding_df.y
repo_blockchain_dataviz_id["z"] = embedding_df.z

In [24]:
repo_blockchain_dataviz_id

,slug,software_type,description,id,x,y,z
0,ethereum/go-ethereum,is_blockchain,Official Go implementation of the Ethereum pro...,151,-0.693368,-0.639979,1.830060
1,freqtrade/freqtrade,is_blockchain,Free open source crypto trading bot,1080,-0.805343,0.576429,0.123971
2,btcsuite/btcd,is_blockchain,An alternative full node bitcoin implementatio...,2664,-0.427841,0.247167,0.741495
3,bitpay/bitcore,is_blockchain,A full stack for bitcoin and blockchain based ...,2854,-0.587301,-0.037816,-0.225613
4,bitcoinj/bitcoinj,is_blockchain,A library for working with Bitcoin,2876,-0.659194,0.993647,0.135701
...,...,...,...,...,...,...,...
246,theJensenLab/phylogician,is_dataviz,Our approach to visualization,123665,2.546024,0.173432,-0.148362
247,davidmtech/old-browser-backup,is_dataviz,JavaScript WebGL 3D maps rendering engine,125701,1.653764,-0.037136,0.333621
248,ajoposor/aoPlotlyAddOn,is_dataviz,a Library of functions to be used to add funct...,126660,2.039129,0.084007,0.445821
249,saramic/data-fe,is_dataviz,a front end for data visualisation experiments,131080,2.137306,0.093315,-0.292292


In [25]:
#repo_blockchain_dataviz_id.to_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_blockchain_dataviz_embedding_3d_pca.csv", index = False)